In [ ]:
import os
import pinn
import tensorflow as tf
from ase.calculators.calculator import all_properties
del all_properties[10:]
all_properties+=[f'{prop}_{extra}' for prop in ['energy', 'forces', 'stress'] for extra in ['avg','std','bias']]
from ase import units
from ase.io import read
from ase.io.trajectory import Trajectory
from ase.md import MDLogger
from tips.bias import EnsembleBiasedCalculator
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen
from glob import glob

tf.get_logger().setLevel('WARNING')
setup = {
  'ensemble': 'nvt', # ensemble
  'T': 340, # temperature in K
  't': 50, # time in ps
  'dt': 0.5, # timestep is fs
  'taut': 100, # thermostat damping in steps
  'taup': 1000, # barastat dampling in steps
  'log-every': 20, # log interval in steps
  'pressure': 1, # pressure in bar
  'compressibility': 4.57e-4, # compressibility in bar^{-1}
  'bias': None,
  'kb': 0,
  'sigma0': 0,
}

In [ ]:
!rm -f asemd.{log,traj}

ensemble=setup['ensemble']
T=float(setup['T'])
t=float(setup['t'])*units.fs*1e3
dt=float(setup['dt'])*units.fs
taut=int(setup['taut'])
taup=int(setup['taup'])
every=int(setup['log-every'])
pressure=float(setup['pressure'])
compressibility=float(setup['compressibility'])

models = glob('/cephyr/users/yunqi/Alvis/pt_pils/models/pils-v5-filtered-pinet-ekf-v3-pre*/model')
calcs = [pinn.get_calc(model) for model in models]
calc = EnsembleBiasedCalculator(calcs, bias='heaviside', kb=10)

atoms = read("../skel/init/hoac-c1im-a32b32i0-rho1.0753.xyz")
atoms.set_calculator(calc)
MaxwellBoltzmannDistribution(atoms, T*units.kB)

if ensemble == 'npt':
    dyn = NPTBerendsen(atoms, timestep=dt, temperature=T, pressure=pressure,
                  taut=dt * taut, taup=dt * taup, compressibility=compressibility)
if ensemble == 'nvt':
    dyn = NVTBerendsen(atoms, timestep=dt, temperature=T, taut=dt * taut)

dyn.attach(
    MDLogger(dyn, atoms, 'asemd.log',stress=True, mode="w"),
    interval=int(every))
dyn.attach(
    Trajectory('asemd.traj', 'w', atoms).write,
    interval=int(every))
dyn.run(int(t/dt))

Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


/cephyr/users/yunqi/Alvis/pinn-tf25/lib/python3.8/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
